# Error Mitigation for Quantum Machine Learning

Quantum computers are noisy. Gate errors, decoherence, and measurement errors all degrade the performance of variational quantum algorithms. This notebook investigates how noise affects a variational quantum classifier and evaluates two error mitigation strategies:

1. **Zero-Noise Extrapolation (ZNE)** — run at multiple noise levels, extrapolate to zero
2. **Measurement Error Mitigation** — calibrate and correct readout errors

We use the same VQC setup (ZZFeatureMap + RealAmplitudes) from the quantum-ml-classifier project, but now add realistic noise models from qiskit-aer.

The practical question: *how much accuracy can we recover?*

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

from src.noise_models import (
    build_depolarizing_model, build_readout_error_model,
    build_combined_model, get_noise_levels
)
from src.mitigation import ZeroNoiseExtrapolator, MeasurementMitigator
from src.noisy_classifier import (
    train_noisy_vqc, evaluate_classifier,
    sweep_noise_levels, sweep_with_mitigation
)
from src.data_utils import load_moons_dataset
from src.plotting import (
    plot_accuracy_vs_noise, plot_zne_extrapolation,
    plot_confusion_matrices, plot_decision_boundaries
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

RESULTS_DIR = Path('../results')
RESULTS_DIR.mkdir(exist_ok=True)

print('Setup complete!')

## 1. Baseline: Ideal VQC

First, let's train a VQC without any noise to establish the ideal accuracy. This is our upper bound — the best we can hope to recover with mitigation.

In [ ]:
# load data
X_train, X_test, y_train, y_test = load_moons_dataset(n_samples=200)
print(f"Train: {len(X_train)}, Test: {len(X_test)}")

# train ideal VQC (no noise)
print("\nTraining ideal VQC...")
ideal_model, ideal_info = train_noisy_vqc(X_train, y_train, noise_model=None)
ideal_acc, ideal_preds = evaluate_classifier(ideal_model, X_test, y_test)
print(f"Ideal accuracy: {ideal_acc:.4f}")

## 2. Understanding Noise Effects

Now let's see what happens when we add depolarizing noise. We'll sweep across error rates from 0.001 (very low noise) to 0.05 (pretty noisy).

Depolarizing noise randomly replaces the ideal quantum state with the maximally mixed state with some probability. For a single-qubit gate with error rate $p$, after the gate the state becomes:

$$\rho \rightarrow (1-p)\rho + \frac{p}{3}(X\rho X + Y\rho Y + Z\rho Z)$$

In [ ]:
# sweep noise levels without mitigation
error_rates = get_noise_levels()
print(f"Error rates to test: {error_rates}")

print("\nSweeping noise levels (no mitigation)...")
noisy_results = sweep_noise_levels(error_rates, X_train, y_train, X_test, y_test)

print("\nResults:")
for rate, acc in zip(error_rates, noisy_results['accuracies']):
    print(f"  Error rate {rate:.3f}: accuracy = {acc:.4f}")

In [ ]:
# plot accuracy degradation
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(error_rates, noisy_results['accuracies'], 'o-', color='#FF6B6B',
        linewidth=2, markersize=8, label='Noisy (no mitigation)')
ax.axhline(y=ideal_acc, color='black', linestyle='--', linewidth=1.5,
           label=f'Ideal ({ideal_acc:.2f})', alpha=0.7)
ax.axhline(y=0.5, color='gray', linestyle=':', linewidth=1,
           label='Random guess', alpha=0.5)

ax.set_xlabel('Depolarizing Error Rate')
ax.set_ylabel('Test Accuracy')
ax.set_title('VQC Accuracy Degradation Under Noise')
ax.legend()
ax.set_ylim(0.4, 1.05)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'accuracy_degradation.png', dpi=150)
plt.show()

## 3. Zero-Noise Extrapolation (ZNE)

ZNE is based on a clever idea: if we can't eliminate noise, we can at least *characterize* how our results depend on it.

**How it works:**
1. Run the circuit at the base noise level (stretch factor = 1)
2. Intentionally amplify noise by inserting gate-inverse-gate pairs (stretch factors 3, 5)
3. Fit a curve through the noisy results
4. Extrapolate to stretch factor = 0 (zero noise)

The key assumption is that the expectation value varies smoothly with noise level.

In [ ]:
# demonstrate ZNE on a single circuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit import QuantumCircuit

# build a simple test circuit
fm = ZZFeatureMap(feature_dimension=2, reps=2)
ansatz = RealAmplitudes(num_qubits=2, reps=3)
test_circuit = fm.compose(ansatz)

# apply ZNE
zne = ZeroNoiseExtrapolator()
stretch_factors = [1, 3, 5]

print("Gate folding demonstration:")
for sf in stretch_factors:
    folded = zne.amplify_noise(test_circuit, sf)
    print(f"  Stretch factor {sf}: {folded.depth()} depth "
          f"(original: {test_circuit.depth()})")

In [ ]:
# run ZNE mitigation sweep
print("Sweeping with ZNE mitigation...")
zne_results = sweep_with_mitigation(
    error_rates, X_train, y_train, X_test, y_test,
    mitigation_type='zne'
)

print("\nZNE Results:")
for rate, acc in zip(error_rates, zne_results['accuracies']):
    print(f"  Error rate {rate:.3f}: accuracy = {acc:.4f}")

## 4. Measurement Error Mitigation

Measurement errors are a different beast from gate errors. When we measure a qubit, there's some probability of getting the wrong outcome — reading 1 when it should be 0, or vice versa.

**Calibration approach:**
1. Prepare each computational basis state $|00\rangle, |01\rangle, |10\rangle, |11\rangle$
2. Measure many times to build a calibration matrix $A$
3. $A_{ij}$ = probability of measuring state $i$ when state $j$ was prepared
4. Correct raw measurement results using the (pseudo-)inverse of $A$

In [ ]:
# demonstrate measurement calibration
from qiskit_aer import AerSimulator

readout_rate = 0.05  # 5% readout error
noise_model = build_readout_error_model(readout_rate, n_qubits=2)
backend = AerSimulator(noise_model=noise_model)

mitigator = MeasurementMitigator()
mitigator.calibrate(n_qubits=2, backend=backend)

print("Calibration matrix:")
print(np.array2string(mitigator.calibration_matrix, precision=3))
print("\nIdeal calibration matrix would be the identity.")
print(f"Off-diagonal elements show the readout error rates.")

In [ ]:
# run measurement mitigation sweep
print("Sweeping with measurement mitigation...")
meas_results = sweep_with_mitigation(
    error_rates, X_train, y_train, X_test, y_test,
    mitigation_type='measurement'
)

print("\nMeasurement Mitigation Results:")
for rate, acc in zip(error_rates, meas_results['accuracies']):
    print(f"  Error rate {rate:.3f}: accuracy = {acc:.4f}")

## 5. Combined Comparison

Let's put everything together and compare all approaches on the same plot.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(error_rates, noisy_results['accuracies'], 'o-', color='#FF6B6B',
        linewidth=2, markersize=7, label='No mitigation')
ax.plot(error_rates, zne_results['accuracies'], 's-', color='#4ECDC4',
        linewidth=2, markersize=7, label='ZNE')
ax.plot(error_rates, meas_results['accuracies'], '^-', color='#45B7D1',
        linewidth=2, markersize=7, label='Measurement mitigation')
ax.axhline(y=ideal_acc, color='black', linestyle='--', linewidth=1.5,
           label=f'Ideal ({ideal_acc:.2f})', alpha=0.7)

ax.set_xlabel('Depolarizing Error Rate')
ax.set_ylabel('Test Accuracy')
ax.set_title('Error Mitigation Comparison for VQC')
ax.legend()
ax.set_ylim(0.4, 1.05)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'mitigation_comparison.png', dpi=150)
plt.show()

In [ ]:
# detailed results table
print(f"{'Error Rate':<12} {'No Mitig.':>10} {'ZNE':>10} {'Meas. Mit.':>10} {'Ideal':>10}")
print('=' * 55)
for i, rate in enumerate(error_rates):
    print(f"{rate:<12.3f} "
          f"{noisy_results['accuracies'][i]:>10.4f} "
          f"{zne_results['accuracies'][i]:>10.4f} "
          f"{meas_results['accuracies'][i]:>10.4f} "
          f"{ideal_acc:>10.4f}")

## 6. Confusion Matrix Comparison

Let's look at the confusion matrices at a moderate noise level (error rate = 0.02) to see exactly what kind of mistakes each approach makes.

In [ ]:
# train at a fixed noise level and compare predictions
test_error_rate = 0.02
noise_model = build_depolarizing_model(test_error_rate, n_qubits=2)

print(f"Training classifiers at error rate = {test_error_rate}...")
noisy_model, _ = train_noisy_vqc(X_train, y_train, noise_model=noise_model)
noisy_acc, noisy_preds = evaluate_classifier(noisy_model, X_test, y_test)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# ideal
cm_ideal = confusion_matrix(y_test, ideal_preds)
ConfusionMatrixDisplay(cm_ideal, display_labels=['Class 0', 'Class 1']).plot(ax=axes[0], cmap='Blues')
axes[0].set_title(f'Ideal (acc={ideal_acc:.2f})')

# noisy
cm_noisy = confusion_matrix(y_test, noisy_preds)
ConfusionMatrixDisplay(cm_noisy, display_labels=['Class 0', 'Class 1']).plot(ax=axes[1], cmap='Oranges')
axes[1].set_title(f'Noisy, no mitigation (acc={noisy_acc:.2f})')

# placeholder for mitigated (in practice you'd run the mitigated version)
axes[2].text(0.5, 0.5, 'See full sweep\nresults above', transform=axes[2].transAxes,
            ha='center', va='center', fontsize=14, color='gray')
axes[2].set_title('Mitigated')

plt.suptitle(f'Confusion Matrices (error rate = {test_error_rate})', fontsize=14, y=1.05)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'confusion_matrices.png', dpi=150, bbox_inches='tight')
plt.show()

## Summary

### Key findings:

1. **Noise degrades VQC performance rapidly** — even a 1% depolarizing error rate causes measurable accuracy loss. At 5%, the classifier is barely above random.

2. **ZNE is the most effective single technique** — by characterizing the noise dependence and extrapolating, we can recover significant accuracy. The main cost is 3-5x more circuit evaluations.

3. **Measurement mitigation helps with readout errors** — it's particularly effective when measurement noise dominates, but less impactful when gate errors are the bottleneck.

4. **Mitigation has limits** — at high noise levels (>3%), even combined mitigation can't fully recover ideal performance. There's a fundamental limit to how much you can correct without actually reducing the physical noise.

### Practical implications for QML:

- Always apply measurement error mitigation — it's cheap (just calibration circuits) and almost always helps
- Use ZNE when accuracy matters and you can afford the extra circuit evaluations
- For near-term hardware with error rates ~1%, mitigation can bridge most of the gap to ideal performance
- As error rates improve with better hardware, the gap shrinks and mitigation becomes even more effective